In [1]:
!pip install transformers
!pip install transformers[sentencepiece]
     

In [9]:
ar_text = """
    عزيزي أمازون، في الأسبوع الماضي قمت بطلب شخصية أوبتيموس برايم للعبة الحركية من متجرك الإلكتروني في ألمانيا.
    للأسف، عندما فتحت الحزمة، اكتشفت برعب أنني تلقيت شخصية ميجاترون بدلاً منها! كعدو مدى الحياة للديسيبتيكونز، آمل أن تفهم محنتي.
    لحل هذه المشكلة، أطالب بتبديل شخصية ميجاترون بشخصية أوبتيموس برايم التي طلبتها.
    في الرسالة مرفق نسخ من سجلاتي المتعلقة بهذا الشراء. أتوقع الاستماع إليكم قريبًا. بكل احترام، بامبلبي.
"""

In [10]:
from transformers import pipeline 
import pandas as pd

### Arabic Text classification

In [14]:
ar_classifier = pipeline(model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",top_k=None)
outputs = ar_classifier(ar_text)
pd.DataFrame(outputs)

,0,1,2
0,"{'label': 'negative', 'score': 0.4802491664886...","{'label': 'positive', 'score': 0.3730726838111...","{'label': 'neutral', 'score': 0.14667823910713..."


### Arabic Named Enecognition 


In [20]:
ar_ner_tagger = pipeline('ner', model='CAMeL-Lab/bert-base-arabic-camelbert-msa-ner')
outputs = ar_ner_tagger(ar_text)
pd.DataFrame(outputs)
     

Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-msa-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,entity,score,index,word,start,end
0,B-ORG,0.987570,2,أما,11,14
1,I-ORG,0.554643,3,##زون,14,17
2,B-MISC,0.968273,11,أوب,52,55
3,B-MISC,0.741880,12,##تيم,55,58
4,I-MISC,0.735298,13,##وس,58,60
5,I-MISC,0.772606,14,برايم,61,66
6,B-LOC,0.997547,24,ألمانيا,104,111
7,B-MISC,0.973556,40,ميجا,172,176
8,B-MISC,0.542810,41,##ترون,176,180
9,B-ORG,0.736683,49,للد,209,212


## Arabic question answering 

In [38]:
ar_reader = pipeline("question-answering",model="ZeyadAhmed/AraElectra-Arabic-SQuADv2-QA")
ar_question = "ماهي عاصمة فلسطين ؟"
ar_context = """
القدس هي عاصمة فلسطين تمتلك مدينة القدس أهميّةً تاريخيّةً ودينيّةً عظيمة
، فهي تُصنّف ضمن أقدم المدن في العالم، وتبلغ مساحتها 126 كيلومتراً مربّعاً، كما تُقدّر الكثافة السكانية في المدينة بـ 7200 نسمة لكلّ كيلومتر مربّع
ومع ذلك فإنّ منزلة المدينة المقدّسة تتجاوز كونها مظهراً جغرافيّاً ومعلماً مكانيّاً، فهي تعبق برائحة التاريخ، والتراث، والهوية المرتبطة بثقافات الفن
، والأدب، والعمارة، ومن هذا المنحى فإنّ القدس لم تكن مجرد عاصمة لدولة فلسطين، بل تجاوزت ذلك لتكون عاصمةً دائمةً للثقافة الإسلامية
"""
outputs = ar_reader(question=ar_question, context=ar_context)
pd.DataFrame([outputs])

,score,start,end,answer
0,0.832639,1,6,القدس


### Arabic Summarization 

In [36]:
summarizer = pipeline("text2text-generation",model="abdalrahmanshahrour/auto-arabic-summarization")
outputs = summarizer(ar_context, max_length=45, clean_up_tokenization_spaces=True)
print(outputs[0]["generated_text"])
     

ومن هذا المنحى فإن القدس لم تكن مجرد عاصمة لدولة فلسطين، بل تجاوزت ذلك لتكون عاصمة دائمة للثقافة الإسلامية.


### Arabic Translation 

In [27]:
from transformers import MarianTokenizer, MarianMTModel
mname = "marefa-nlp/marefa-mt-en-ar"
tokenizer = MarianTokenizer.from_pretrained(mname)
model = MarianMTModel.from_pretrained(mname)

# English Sample Text
input = "President Putin went to the presidential palace in the capital, Kiev"

translated_tokens = model.generate(**tokenizer.prepare_seq2seq_batch([input], return_tensors="pt"))
translated_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]

# translated Arabic Text
print(translated_text)
# ذهب الرئيس پوتن إلى القصر الرئاسي في العاصمة كييڤ


tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/801k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3992: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


['ذهب الرئيس پوتن إلى القصر الرئاسي في العاصمة كييڤ']


### arabic Text Generation

In [45]:
#Requirements: !pip install transformers
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("MIIB-NLP/Arabic-question-generation")
tokenizer = AutoTokenizer.from_pretrained("MIIB-NLP/Arabic-question-generation")

def get_question(context,answer):
  text="context: " +context + " " + "answer: " + answer + " </s>"
  text_encoding = tokenizer.encode_plus(
      text,return_tensors="pt"
  )
  model.eval()
  generated_ids =  model.generate(
    input_ids=text_encoding['input_ids'],
    attention_mask=text_encoding['attention_mask'],
    max_length=64,
    num_beams=5,
    num_return_sequences=1
  )
  return tokenizer.decode(generated_ids[0],skip_special_tokens=True,clean_up_tokenization_spaces=True).replace('question: ',' ')

context="الثورة الجزائرية أو ثورة المليون شهيد، اندلعت في 1 نوفمبر 1954 ضد المستعمر الفرنسي ودامت 7 سنوات ونصف. استشهد فيها أكثر من مليون ونصف مليون جزائري"
answer =" 7 سنوات ونصف"

get_question(context,answer)

#output : question="كم استمرت الثورة الجزائرية؟ "


config.json:   0%|          | 0.00/780 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.76M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

' كم استمرت الثورة الجزائرية؟ '